In [40]:
from dotenv import load_dotenv
import os

load_dotenv(".env")

True

In [9]:
# langchain 라이브러리 설치
!pip install langchain-openai

## 환경변수 로드

In [42]:
api_key = os.getenv("OPENAI_API_KEY")

## LLM 초기화
- openAI

In [21]:
from langchain_openai import OpenAI

In [23]:
llm = OpenAI(api_key=api_key)

## OpenAI API를 직접 이용하여 응답 생성 파이프라인 생성

In [29]:
import openai
from typing import List

In [31]:
client = openai.OpenAI()

#### 가장 간단한 형태

In [33]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{"role" : "user", "content" : "안녕하세요!"}]
)

In [35]:
print(response.choices[0].message.content)

안녕하세요! 어떻게 도와드릴까요?


#### 함수로 분리

In [43]:
prompt_format = "주제 {topic}에 대해 짧은 설명을 해주세요!"

def call_chat_model(messages: List[dict]):
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )

    return response.choices[0].message.content

def invoke_chain(topic: str):
    prompt_value = prompt_format.format(topic=topic)

    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

In [45]:
# 더블딥을 주제로 설명 요청
invoke_chain("더블딥")

'"더블딥(double dip)"은 경제학에서 사용되는 용어로, 경기 침체가 한 번 발생한 후에 간헐적으로 회복을 보이다가 다시 침체에 빠지는 상황을 가리킵니다. 이 현상은 V자형 회복이 아닌 W자형 회복으로 나타나며, 즉 경기가 두 번 하락하는 패턴을 보입니다. 더블딥은 종종 심각한 경제적 불확실성과 어려움을 동반하며, 통화 정책과 재정 정책의 효과성에 대한 논쟁을 불러일으킬 수 있습니다.'

## 랭체인 프레임워크를 활용한 OpenAI 응답

In [48]:
# 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

#### 랭체인 단일 출력

In [50]:
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대해 짧은 설명을 해주세요."
)

In [52]:
output_parser = StrOutputParser()

model = ChatOpenAI(model="gpt-4o-mini")

In [54]:
chain = (
    {"topic": RunnablePassthrough()} # 입력받은 주제르 그대로 통과
    | prompt # 프롬프트 템플릿 적용 
    | model # 모델을 사용해 응답 생성
    | output_parser # 응답을 문자열로 파싱
)

In [56]:
chain.invoke("더블딥")

'더블딥(Double Dip)은 경제학에서 사용되는 용어로, 경제가 한 번 회복된 후 다시 침체에 빠지는 상황을 설명합니다. 일반적으로 경기 침체가 발생한 후, 일시적인 회복세를 보이다가 다시 부진으로 돌아가는 경우를 가리킵니다. 이는 주로 소비자 신뢰의 부족, 고용 시장의 회복 미비, 혹은 외부 경제적 요인 등에 의해 발생할 수 있습니다. 더블딥 경제는 경기 회복이 불확실하고 지속적이지 않을 때 투자자나 기업에게 큰 위험 요소로 작용합니다.'

#### 랭체인 배치 출력

In [59]:
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대해 짧은 설명을 해주세요."
)

In [63]:
model, parser = ChatOpenAI(model="gpt-4o-mini"), StrOutputParser()

chain = prompt | model | parser

In [65]:
chain.batch([
    {"topic": "더블딥"},
    {"topic": "인플레이션"}
])

['더블 딥(Double Dip)은 경제학에서 사용되는 용어로, 경기 침체가 발생한 후 경제가 일시적으로 회복되다가 다시 다시 침체에 빠지는 현상을 설명합니다. 즉, 경제가 처음에는 정상으로 돌아오는 듯하지만, 그 회복이 지속되지 않고 다시 하락세로 돌아서는 상황을 나타냅니다. 주로 주식 시장, 실업률, 소비자 신뢰 등 여러 경제 지표에서 관찰될 수 있으며, 이런 현상은 경제 회복의 불안정성을 시사합니다.',
 '인플레이션은 경제에서 일반적인 가격 수준이 지속적으로 상승하는 현상을 의미합니다. 즉, 시간이 지남에 따라 화폐의 구매력이 감소하게 됩니다. 인플레이션은 다양한 요인에 의해 발생할 수 있으며, 소비자 물가 지수(CPI)와 같은 지표로 측정됩니다. 인플레이션이 적정 수준일 경우 경제 성장에 긍정적인 영향을 미칠 수 있지만, 너무 높은 인플레이션은 경제 불안정과 생활비 상승을 초래할 수 있습니다. 따라서 정부와 중앙은행은 인플레이션을 관리하기 위해 다양한 통화 정책을 시행합니다.']

#### 스트리밍 방식의 출력

In [71]:
for token in chain.stream({"topic": "더블딥"}):
    print(token, end="", flush=True)

"더블딥(Double Dip)"은 경제학에서 주로 사용되는 용어로, 경기 침체가 두 차례에 걸쳐 발생하는 상황을 의미합니다. 첫 번째 경기 침체가 발생한 후, 경제가 잠시 회복되는 듯 보이다가 다시 침체에 빠지는 경우를 지칭합니다. 이러한 현상은 소비자 신뢰와 기업 투자에 영향을 미치며, 경제 전반에 불확실성을 증가시킬 수 있습니다. 더블딥은 일반적으로 급격한 진입과 회복이 동시에 나타나는 경우에 관찰됩니다.

#### 러너블을 체인으로 연결

In [78]:
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해주세요: {answer}")

In [80]:
composed_chain = {"answer" : chain} | analysis_prompt | model | StrOutputParser()

for token in composed_chain.stream({"topic": "더블딥"}):
    print(token, end="", flush=True)

"Double Dip" is a term primarily used in economics to describe a situation where a recession occurs, followed by a brief recovery that does not sustain itself, leading to a second recession. In other words, it refers to the phenomenon where the economy shows some signs of recovery after a downturn, only to decline again. This can generally happen due to various factors such as low consumer confidence, decreased business investment, and instability in the labor market. A double dip can increase uncertainty in the economy and complicate the recovery process.

#### 체인 병렬 연결

In [85]:
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(model="gpt-4o-mini")

kor_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧은 설명을 한국어로 해주세요.")
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧은 설명을 영어로 해주세요.")
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)

result = parallel_chain.invoke({"topic": "더블딥"})

print("한글 설명: ", result['kor'])
print("영어 설명: ", result['eng'])

한글 설명:  더블딥(Double Dip)은 경제학에서 사용되는 용어로, 경기침체 후 잠시 회복된 뒤 다시 경제가 침체되는 현상을 의미합니다. 즉, 첫 번째 침체 후에 일시적으로 경제가 개선되지만, 다시 두 번째 침체가 발생하여 경기 회복이 이어지지 않는 상황을 가리킵니다. 이러한 현상은 고용, 소비, 투자 등 여러 경제 지표에 부정적인 영향을 미칠 수 있습니다.
영어 설명:  A double dip refers to a situation where an economy experiences two separate periods of recession within a relatively short time frame. After a mild recovery from an initial downturn, the economy falls back into recession, resembling a "W" shape in economic graphs. This can be caused by various factors, including weak consumer confidence, lack of job growth, or external shocks, and often complicates recovery efforts.


## 프롬프트 템플릿

#### 문자열 프롬프트 템플릿

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_templeate = PromptTemplate.from_template("주제 {topic} 에 대해 금융 관련 짧은 조언을 해주세요")

prompt_templeate.invoke({"topic": "투자"})

StringPromptValue(text='주제 투자 에 대해 금융 관련 짧은 조언을 해주세요')

#### 챗 프롬프트 템플릿

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic} 에 대해 금융 관련 조언을 해주세요.")
])

prompt_template.invoke({"topic": "주식"})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식 에 대해 금융 관련 조언을 해주세요.', additional_kwargs={}, response_metadata={})])

#### 메시지 자리 표시자

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content = "안녕하세요!")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

#### 퓨샷 프롬프트
- 대규모 언어 모델이 더 나은 성능을 발휘하도록 몇 가리 예제 입력과 출력을 제공하는 방식
- 퓨샷 기술은 모델에 몇 가지 예제를 제시하여, 모델이 더욱 정확하고 일관된 결과를 생성하도록 유도합니다.

In [19]:
from langchain_core.prompts import PromptTemplate

# 질문과 답변을 포맷하는 프롬프트 템플릿 정의
example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

In [29]:
# 퓨샷 예제 목록 생성
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
    후속 질문이 필요한가요: 네
    후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변: 주식 투자의 평균 수익률은 연 7% 입니다.
    후속 질문: 예금의 평균 이자율은 얼마인가요?
    중간 답변: 예금 평균 이자율은 연 1% 입니다.
    따라서 최종 답변은: 주식 투자
    """
    },
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
    "answer": """
    후속 질문이 필요한가요: 네
    후속 질문: 부동산 투자의 안정성은 어떤 요소에 따라 결정되나요?
    중간 답변: 부동산 투자는 지역 경기와 정부 정책의 영향을 많이 받아 변동성이 존재합니다.
    후속 질문: 채권 투자의 안정성은 어떤가요?
    중간 답변: 채권은 고정 수익을 제공하며, 정부나 우량 기업이 발행한 채권은 비교적 안정적인 편입니다.
    따라서 최종 답변은: 채권
    """
    }
]

In [31]:
print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
    후속 질문이 필요한가요: 네
    후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변: 주식 투자의 평균 수익률은 연 7% 입니다.
    후속 질문: 예금의 평균 이자율은 얼마인가요?
    중간 답변: 예금 평균 이자율은 연 1% 입니다.
    따라서 최종 답변은: 주식 투자
    


In [33]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "질문: {input}",
    input_variables=["input"]
)

print(prompt.invoke({"input" : "부동산 투자의 장점은 무엇인가?"}).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
    후속 질문이 필요한가요: 네
    후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변: 주식 투자의 평균 수익률은 연 7% 입니다.
    후속 질문: 예금의 평균 이자율은 얼마인가요?
    중간 답변: 예금 평균 이자율은 연 1% 입니다.
    따라서 최종 답변은: 주식 투자
    

질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
    후속 질문이 필요한가요: 네
    후속 질문: 부동산 투자의 안정성은 어떤 요소에 따라 결정되나요?
    중간 답변: 부동산 투자는 지역 경기와 정부 정책의 영향을 많이 받아 변동성이 존재합니다.
    후속 질문: 채권 투자의 안정성은 어떤가요?
    중간 답변: 채권은 고정 수익을 제공하며, 정부나 우량 기업이 발행한 채권은 비교적 안정적인 편입니다.
    따라서 최종 답변은: 채권
    

질문: 부동산 투자의 장점은 무엇인가?


#### 예제 선택기 사용

In [44]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 예제 선택기 초기화
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # 사용할 예제 목록
    OpenAIEmbeddings(api_key=api_key), # 임베딩 생성에 사용하는 클래스
    Chroma, # 임베딩을 저장하고 유사도 검색을 수행하는 벡터 저장소 클래스
    k = 1 # 선택할 예제의 수
)

In [46]:
question = "부동산 투자의 장점은 무엇인가?"
selected_examples = example_selector.select_examples({"question": question})

In [50]:
print(f"입력질문: {question}")
for example in selected_examples:
    print("\n")
    print("# 입력과 가장 유사한 예제:")
    for k, v in reversed(example.items()):
        print(f"{k}, {v}")

입력질문: 부동산 투자의 장점은 무엇인가?


# 입력과 가장 유사한 예제:
answer, 
    후속 질문이 필요한가요: 네
    후속 질문: 부동산 투자의 안정성은 어떤 요소에 따라 결정되나요?
    중간 답변: 부동산 투자는 지역 경기와 정부 정책의 영향을 많이 받아 변동성이 존재합니다.
    후속 질문: 채권 투자의 안정성은 어떤가요?
    중간 답변: 채권은 고정 수익을 제공하며, 정부나 우량 기업이 발행한 채권은 비교적 안정적인 편입니다.
    따라서 최종 답변은: 채권
    
question, 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?


#### 퓨샷 프롬프트를 실제 AI 모델과 함께 사용하는 코드 예시

In [53]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template = "질문: {question}\n답변: {answer}"
)

In [55]:
# 퓨샷 프롬프트 생성
prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "다음은 금융 관련 짋문과 답변의 예입니다:",
    suffix = "질문: {input}\n답변:",
    input_variables = ["input"]
)

In [57]:
model = ChatOpenAI(model_name = "gpt-4o-mini")

chain = prompt | model
response = chain.invoke({"input" : "부동산 투자의 장점은 무엇인가?"})
print(response.content)

부동산 투자의 장점에는 여러 가지가 있습니다. 

1. **자산의 가치 상승**: 부동산은 시간이 지남에 따라 가치가 상승할 가능성이 높아, 장기적인 투자로 유리합니다.
2. **임대 수익**: 부동산은 임대를 통해 지속적인 현금 흐름을 생성할 수 있어 수익성을 높입니다.
3. **세금 혜택**: 부동산 투자자는 정부에서 제공하는 다양한 세금 공제와 혜택을 받을 수 있습니다.
4. **인플레이션 헤지**: 부동산은 인플레이션에 대해 상대적으로 안전한 자산으로 여겨져, 물가 상승에 대한 방어 수단이 될 수 있습니다.
5. **자산의 다양화**: 포트폴리오에 부동산을 포함하면 투자 자산의 다양화를 통해 리스크를 분산할 수 있습니다.

후속 질문이 필요할까요?


## 출력 파서

#### pydanticOutputParser

- PydanticOutputParser : AI의 출력을 Pydantic 모델에 맞게 구조화된 데이터로 변환하며, 일관된 형식과 데이터 검증을 제공
- Pydantic : Python에서 데이터 검증과 모델링을 위한 라이브러리, BaseModel과 Field를 사용하여 데이터 구조를 정의하고, model_validator를 이용하여 입력데이터를 검증 가능

In [63]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

In [67]:
model = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0.0)

In [69]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description = "금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description = "질문을 해결하기 위한 금융 답변")

    @model_validator(mode = "before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup", "")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return values

In [87]:
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template = (
        "다음 금융 관련 질문을 '?'로 끝나는 형태로 구성한 뒤, 그에 대한 조언을 해주세요.\n"
        "{format_instructions}\n질문: {query}\n"
    ),
    input_variables = ["query"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

In [89]:
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있게 질문하라."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

setup='부동산 투자 시 어떤 요소를 고려해야 할까요?' advice='부동산 투자 시 위치, 시장 동향, 자금 조달 방법, 세금 혜택, 그리고 장기적인 가치 상승 가능성을 고려해야 합니다. 또한, 전문가의 조언을 듣고, 충분한 시장 조사를 통해 신중하게 결정하는 것이 중요합니다.'


#### JsonOutputParser 이용

In [100]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=FinancialAdvice)

prompt = PromptTemplate(
    template = (
        "다음 금융 관련 질문을 '?'로 끝나는 형태로 구성한 뒤, 그에 대한 조언을 해주세요.\n"
        "{format_instructions}\n질문: {query}\n"
    ),
    input_variables = ["query"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

In [102]:
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있게 질문하라."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

{'setup': '부동산 투자 시 어떤 요소를 고려해야 할까요?', 'advice': '부동산 투자 시 위치, 시장 동향, 자금 조달 방법, 세금 혜택, 그리고 장기적인 가치 상승 가능성을 고려해야 합니다. 또한, 전문가의 조언을 듣고, 충분한 시장 조사를 통해 신중하게 결정하는 것이 중요합니다.'}
